In [1]:
__author__ = 'Camilla Pacifici, Brett Morris, Benjamin Weaver <benjamin.weaver@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>'
__version__ = '20231010' # yyyymmdd
__datasets__ = ['desi_edr']  
__keywords__ = ['sparcl', 'jdaviz', 'specutils', 'spectroscopy', 'desi spectra', 'tutorial']

# SPARCL + Jdaviz

### Table of contents
* [Goals](#goals)
* [Introduction](#intro)
* [Disclaimer & attribution](#disclaimer)
* [Imports and setup](#imports)
* [Search for spectra in SPARCL](#searchsparcl)
* [Retrieve spectra from SPARCL and prepare plot data](#retrievesparcl)
* [Plot spectra using Jdaviz](#plotjdaviz)

<a class="anchor" id="goals"></a>
## Goals

* Search for and obtain spectra from DESI EDR using the [NOIRLab SPARCL service](https://astrosparcl.datalab.noirlab.edu)
* Plot spectra using the [Jdaviz](https://jdaviz.readthedocs.io/en/latest/index.html) data analysis visualization tool

<a class="anchor" id="intro"></a>
## Introduction

This notebook demonstrates how to find and retrieve spectroscopic data for certain objects from the DESI EDR data set using [SPARCL](https://astrosparcl.datalab.noirlab.edu) (SPectra Analysis and Retrievable Catalog Lab) and display an interactive plot of an object's spectrum using [Jdaviz](https://jdaviz.readthedocs.io/en/latest/index.html).

This notebook is based on [an example](https://github.com/camipacifici/jdaviz/blob/concept-sparcl/notebooks/concepts/specviz_sparcl.ipynb) originally developed by [Camilla Pacifici](https://github.com/camipacifici). Additional assistance was provided by
[Brett Morris](https://github.com/bmorris3). 

Packages:

* [SPARCL](https://astrosparcl.datalab.noirlab.edu)
* [Jdaviz](https://jdaviz.readthedocs.io/en/latest/index.html)
* [jupyterlab-sidecar](https://github.com/jupyter-widgets/jupyterlab-sidecar)

<a class="anchor" id="disclaimer"></a>
## Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* SPARCL: https://astrosparcl.datalab.noirlab.edu/sparc/acknowledgments/

* Jdaviz: https://jdaviz.readthedocs.io/en/latest/index_citation.html

<a class="anchor" id="imports"></a>
## Imports and setup

In [1]:
# SPARCL import
from sparcl.client import SparclClient
# Jdaviz import
from jdaviz import Specviz
# Jupyter
import ipywidgets as widgets
from IPython.display import display
# Specutils import
from specutils import Spectrum1D
# Astropy import
import astropy.units as u
from astropy.nddata import InverseVariance, StdDevUncertainty

ModuleNotFoundError: No module named 'jdaviz'

## Set up the SPARCL client

In [3]:
client = SparclClient()
client

(sparclclient:1.2.0, api:9.0, https://astrosparcl.datalab.noirlab.edu/sparc, verbose=False, connect_timeout=1.1, read_timeout=5400.0)

#### View the full list of fields for the DESI EDR data set that can be obtained from SPARCL

In [4]:
desi_fields = client.get_all_fields(dataset_list=['DESI-EDR'])
print(desi_fields)

['bgs_target', 'chi2', 'cmx_target', 'coadd_fiberstatus', 'coadd_numexp', 'coadd_numnight', 'coadd_numtile', 'coeff', 'data_release', 'datasetgroup', 'dateobs', 'dateobs_center', 'dec', 'deltachi2', 'desi_target', 'exptime', 'fa_target', 'fa_type', 'firstnight', 'flux', 'healpix', 'instrument', 'ivar', 'lastnight', 'mask', 'mean_delta_x', 'mean_delta_y', 'mean_fiber_dec', 'mean_fiber_ra', 'mean_mjd', 'mean_psf_to_fiber_specflux', 'model', 'mws_target', 'ncoeff', 'npixels', 'numobs_init', 'objtype', 'obsconditions', 'plate_dec', 'plate_ra', 'pmdec', 'pmra', 'priority_init', 'program', 'ra', 'redshift', 'redshift_err', 'redshift_warning', 'ref_epoch', 'rms_delta_x', 'rms_delta_y', 'scnd_target', 'site', 'sparcl_id', 'specid', 'specprimary', 'spectype', 'spgrpval', 'std_fiber_dec', 'std_fiber_ra', 'subpriority', 'subtype', 'survey', 'sv1_bgs_target', 'sv1_desi_target', 'sv1_mws_target', 'sv1_scnd_target', 'sv2_bgs_target', 'sv2_desi_target', 'sv2_mws_target', 'sv2_scnd_target', 'sv3_bgs_t

<a class="anchor" id="searchsparcl"></a>
## Search for spectra in SPARCL
Using the `client.find()` method, we obtain the following fields from the DESI EDR data set using SPARCL:  
- `sparcl_id` : Universally Unique Identifier for spectrum in SPARCL
- `ra` : Right Ascension in degrees
- `dec` : Declination in degrees
- `spectype` : Spectral type of the object (STAR, GALAXY, or QSO)
- `subtype` : Spectral subtype
- `specid` : Dataset-specific spectrum identifier (may not be unique)
- `redshift` : Measured redshift
- `redshift_err` : Uncertainty on the measured redshift

And apply the following constraints:
- `2.0 < redshift < 5.0`
- `redshift_warning = 0` (Redshift warning bitmask measured by Redrock)
- `specprimary = 1` (Boolean flag (True/False) for the primary coadded spectrum for a given target object)
- `spectype = QSO`
- `data_release = DESI EDR`

In [5]:
outfields = ['sparcl_id', 'ra', 'dec', 'spectype', 'subtype', 'specid',
             'redshift', 'redshift_err',]
constraints = {'redshift': [2.0, 5.0],
               'redshift_warning': [0],
               'specprimary': [1],
               'spectype': ['QSO'],
               'data_release': ['DESI-EDR'],
               }

In [6]:
found = client.find(outfields=outfields,
                    constraints=constraints,
                    limit=50)

Create `DESIID` from the RA and Dec of each record:

In [7]:
for record in found.records:
    if record['_dr'] == 'DESI-EDR':
        record['DESIID'] = f"DESI J{record['ra']:08.4f}{record['dec']:+08.4f}"
metadata = sorted(found.records, key=lambda x: x['DESIID'])
ids = [m['sparcl_id'] for m in metadata]

<a class="anchor" id="retrievesparcl"></a>
## Retrieve spectra from SPARCL and prepare plot data
Here we define a function that:
1. Uses the `client.retrieve()` method from SPARCL to retrieve spectra for the list of IDs from DESI EDR.
2. Creates a Spectrum1D object from the retrieved data.
3. Loads the spectrum, metadata, and model.

In [8]:
def on_selected(b):
    key = entries[b['new']][0]
    result = client.retrieve(uuid_list=[ids[b['new']]],
                             include=['specid',
                                      'survey',
                                      'ra',
                                      'dec',
                                      'redshift',
                                      'redshift_err',
                                      'flux',
                                      'ivar',
                                      'wavelength',
                                      'mask',
                                      'model'])
    if key not in spectra:
        spectra[key] = (Spectrum1D(spectral_axis=result.records[0]['wavelength']*u.AA,
                                   flux=result.records[0]['flux']*specunit,
                                   uncertainty=InverseVariance(result.records[0]['ivar']*(specunit**-2)).represent_as(StdDevUncertainty),
                                   mask=result.records[0]['mask'],
                                   redshift=result.records[0]['redshift']),
                        Spectrum1D(spectral_axis=result.records[0]['wavelength']*u.AA,
                                   flux=result.records[0]['model']*specunit))
    if 'old' in b:
        #
        # Remove the previous spectrum.
        #
        data0 = specviz.app.data_collection[0]
        data1 = specviz.app.data_collection[1]
        specviz.app.data_collection.remove(data0)
        specviz.app.data_collection.remove(data1)
    #
    # Load the spectrum.
    #
    specviz.load_data(spectra[key][0], data_label=key + ' Data')
    opt = specviz.plugins['Plot Options']
    opt.layer = key + ' Data'
    opt.line_color.value = '#000000'  # Black
    opt.line_as_steps = True
    opt.uncertainty_visible = True
    #
    # Add metadata.
    #
    meta0 = specviz.app.data_collection[0].meta
    meta0['RA'] = result.records[0]['ra']
    meta0['Dec'] = result.records[0]['dec']
    meta0['redshift'] = result.records[0]['redshift']
    meta0['redshift uncertainty'] = result.records[0]['redshift_err']
    #
    # Load the model.
    #
    specviz.load_data(spectra[key][1], data_label=key + ' Model')
    opt = specviz.plugins['Plot Options']
    opt.layer = key + ' Model'
    opt.line_color.value = '#FF0000'  # Red
    opt.line_as_steps = True
    #
    # This is a workaround for a bug where metadata is not initially displayed.
    #
    m = specviz.plugins['Metadata']
    m.dataset = key + ' Model'
    m.dataset = key + ' Data'

<a class="anchor" id="plotjdaviz"></a>
## Plot spectra using Jdaviz
Run the cell below to open the interactive plot in a new panel on the right. Information on the Toolbar options can be found in the [Jdaviz User Guide](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html).

This cell will also display a drop-down menu that controls the spectrum that will be displayed in the interactive plot.

In [ ]:
specunit = u.Unit('10-17 erg cm-2 s-1 AA-1')
spectra = dict()  # This will cache the retrieved spectra.
entries = [(m['DESIID'], i) for i, m in enumerate(metadata)]
select = widgets.Dropdown(options=entries, value=0, description='Spectrum:',)
output = widgets.Output()
specviz = Specviz()

#
# Trigger the download if this is the very first spectrum.
#
on_selected({'new': 0})
#
# Connect the function action to the dropdown menu.
#
select.observe(on_selected, names='value')
#
# Open the display in a separate tab.
#
specviz.show('sidecar:split-right')
#
# Display the drop-down menu.
#
display(select, output)